Задание 1.
Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [9]:
import requests

In [10]:
import pandas as pd

In [11]:
from bs4 import BeautifulSoup

In [12]:
import time

In [14]:
import re

In [61]:
# определяем список слов, которые нам интересны
KEYWORDS = ['python', 'парсинг', 'javascript']

In [62]:
# получаем страницу с самыми свежими статьями
url = 'https://habr.com/ru/all/'
req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')

In [63]:
# извлекаем превью статьи из страницы
post_previews = soup.find_all('article', class_='post post_preview')

In [66]:
post_preview.find(class_ = "btn btn_x-large btn_outline_blue post__habracut-btn").get('href')

'https://habr.com/ru/company/wd/blog/522804/#habracut'

In [91]:
# для текст превью делаем проверку на вхождение любого слова в требуемый список KEYWORDS, если выполняется - записываем итоговый список
def get_all_previews(url):
    habr_articles = []
    for post_preview in post_previews:
        for class_ in ['post__text post__text-html post__text_v1', 
                       'post__text post__text-html post__text_v2']:
            if post_preview.find('div', class_= class_):
                teg = post_preview.find(class_=class_).text.lower()
                teg_split = re.split(' |, |.\r\n',teg)
                if any(keyword.lower() in teg_split for keyword in KEYWORDS):
                    date = post_preview.find(class_='post__time').text
                    title = post_preview.find(class_='post__title_link').text
                    link = post_preview.find(class_ = "btn btn_x-large btn_outline_blue post__habracut-btn").get('href')
                    row = {'date': date, 'title': title, 'link': link}
                    habr_articles.append(row)
    return habr_articles

# Преобразуем в DF
habr_articles_python = pd.DataFrame(get_all_previews(url))
habr_articles_python
# habr_articles_python.link

,date,title,link
0,сегодня в 14:43,Эффектное программирование. Часть 1: итераторы...,https://habr.com/ru/post/522864/#habracut


In [92]:
#Добавляем текст статьи (доп задание)
for link in habr_articles_python.link:
    req1 = requests.get(str(link))
    soup = BeautifulSoup(req1.text, 'html.parser')
    text_article = soup.find(class_='post__text post__text-html post__text_v1').text
    habr_articles_python['article'] = text_article.strip()
habr_articles_python

,date,title,link,article
0,сегодня в 14:43,Эффектное программирование. Часть 1: итераторы...,https://habr.com/ru/post/522864/#habracut,Javascript на данный момент является самым поп...


Задание 2.
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [52]:
emails = ["alvin@ngs.ru"]
Headers = {
    'Host': 'identityprotection.avast.com',
    'Connection': 'keep-alive',
    'Content-Length': '35',
    'Accept': 'application/json, text/plain, */*',
    'Vaar-Version': '0',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Content-Type': 'application/json;charset=UTF-8',
    'Origin': 'https://www.avast.com',
    'Sec-Fetch-Site': 'same-site',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Dest': 'empty',
    'Referer': 'https://www.avast.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7'
}
result_json = requests.post('https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data', json = {"emailAddresses": emails}, headers = Headers).json()
result_json

{'breaches': {'13094': {'site': 'myheritage.com',
   'recordsCount': 110042353,
   'description': 'In October 2017, a customer database belonging to online genealogy platform MyHeritage was breached. The stolen records included email addresses and SHA-1 hashed passwords, as well as sensitive PII. The data has been sold on at least one dark web market and is also being privately shared amongst several criminal groups.',
   'publishDate': '2017-11-04T00:00:00Z'},
  '12': {'site': 'vk.com',
   'recordsCount': 91262655,
   'description': "Popular Russian social networking platform VKontakte was breached in late 2012. Over 100 million clear-text passwords were compromised in the breach. Breached credential sets included victims' e-mail addresses, passwords, dates of birth, phone numbers and location details. The credential set was advertised on a dark web marketplace as of June 2016 for a price of one bitcoin. ",
   'publishDate': '2016-10-29T00:00:00Z'}},
 'data': {'myheritage.com': {'alvi

In [53]:
res = []
for email in emails:
    for breach in result_json['summary'][email]['breaches']:
        res.append({'email': str(email),'date':str(result_json['breaches'][str(breach)]['publishDate']),
                     'site':str(result_json['breaches'][str(breach)]['site']),
                     'description':str(result_json['breaches'][str(breach)]['description'])})
pd.DataFrame(res)

,email,date,site,description
0,alvin@ngs.ru,2017-11-04T00:00:00Z,myheritage.com,"In October 2017, a customer database belonging..."
1,alvin@ngs.ru,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...


Дополнительная часть (необязательная)
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>